- https://colab.research.google.com/drive/1F_RNcHzTfFuQf-LeKvSlud6x7jXYkG31#scrollTo=goRmGIRI5cfC

# setup

http://localhost:8080/notebooks/git/product-category/notebooks/prep_20210304A1.ipynb

In [1]:
prfx_prp = 'prep_20210304A1'

In [2]:
HOME = "/data/git/product-category"
p_out = f'{HOME}/data/transformer_20210306A1'
!mkdir -p {p_out}

In [3]:
sz = int(1e4)
DATA2USE = f'{HOME}/data/data_sample_{sz}__{prfx_prp}.csv'

# eda 

In [4]:
import pandas as pd
import numpy as np
from collections import Counter

In [5]:
%%time
# df = pd.read_csv(f'../data/data__{prfx_prp}.csv')
# df = pd.read_csv(f'{HOME}/data/data_sample__{prfx_prp}.csv')
df = pd.read_csv(DATA2USE, nrows=1000)
print(df.shape)
df.sample(3)

(1000, 9)
CPU times: user 17.3 ms, sys: 4.25 ms, total: 21.5 ms
Wall time: 20.1 ms


,category,description,title,brand,feature,asin,domain,txt,is_validation
583,Pet Supplies|Cats|Beds & Furniture,The Penn Plax Cat Scratching Post is has a uni...,Penn Plax Cat Scratching Post Sisal with Swatt...,Penn Plax,"Putting the fun into functionality, the Penn P...",B0097GY7IO,Pet_Supplie,Penn Plax Cat Scratching Post Sisal with Swatt...,1.0
461,Books|Christian Books & Bibles|Protestantism,NaN,Beyond 2012: What the Real Prophets Are Saying,Visit Amazon's Julia C. Loren Page,NaN,0977637026,Book,Beyond 2012: What the Real Prophets Are Saying...,0.0
668,"Clothing, Shoes & Jewelry|Costumes & Accessori...",The perfect foxy tail & furry eared headband s...,Pawstar Brown Furry Fox Tail & ERAS Costume Set,Pawstar,"Faux Fur\nOfficial Pawstar Product\n23"" Long t...",B0160N7ZWQ,Clothing_Shoes_and_Jewelry,Pawstar Brown Furry Fox Tail & ERAS Costume Se...,0.0


In [6]:
df.txt.notna().mean()

0.938

In [7]:
MIN_CNT = 50
dmn2cnt = Counter(df.domain.value_counts().to_dict())
i2dmn = sorted(dmn2cnt.keys())
dmn2i = {v:k for k,v in enumerate(i2dmn)}
cat2cnt = Counter((j for i in df.category.apply(lambda x: x.split('|')) for j in i))
i2cat = sorted(k for k,v in cat2cnt.items() if v>50)
cat2i = {v:k for k,v in enumerate(i2cat)}

print("len(i2dmn), len(i2cat)", len(i2dmn), len(i2cat))
print("|".join(i2dmn))
print()
print("|".join(i2cat))

len(i2dmn), len(i2cat) 22 9
Appliance|Arts_Crafts_and_Sewi|Automotive|Book|CDs_and_Vinyl|Cell_Phones_and_Accessorie|Clothing_Shoes_and_Jewelry|Electronic|Grocery_and_Gourmet_Food|Home_and_Kitche|Industrial_and_Scientific|Kindle_Store|Movies_and_TV|Musical_Instrument|Office_Product|Patio_Lawn_and_Garde|Pet_Supplie|Software|Sports_and_Outdoor|Tools_and_Home_Improvement|Toys_and_Game|Video_Game

Automotive|Books|Clothing|Clothing, Shoes & Jewelry|Electronics|Home & Kitchen|Men|Sports & Outdoors|Women


# dataset

In [8]:
from argparse import ArgumentParser
import pytorch_lightning as pl
from transformers.optimization import AdamW

from torch.utils.data import TensorDataset, DataLoader
from transformers import AutoTokenizer
import torch

def mk_tensors(txt, tokenizer, max_seq_length):
    tok_res = tokenizer(
        txt, truncation=True, padding='max_length', max_length=max_seq_length
    )
    input_ids = tok_res["input_ids"]
    attention_mask = tok_res["attention_mask"]
    input_ids = torch.tensor(input_ids)
    attention_mask = torch.tensor(attention_mask)
    return input_ids, attention_mask

def mk_ds(txt, tokenizer, max_seq_length, ys):
    input_ids, attention_mask = mk_tensors(txt, tokenizer, max_seq_length)
    return TensorDataset(input_ids, 
                         attention_mask, 
                         torch.tensor(ys)) 

class PCDataModule(pl.LightningDataModule):
    def __init__(self, 
                 model_name_or_path, 
                 max_seq_length, 
                 min_products_for_category,
                 train_batch_size,
                 val_batch_size,
                 data_file_path=None,
                 dataframe=None):
        super().__init__()
        self.data_file_path = data_file_path
        self.dataframe = dataframe
        self.min_products_for_category = min_products_for_category
        self.model_name_or_path = model_name_or_path
        self.max_seq_length = max_seq_length
        self.train_batch_size = train_batch_size
        self.val_batch_size = val_batch_size
        self.num_classes = None
      
    def prepare_data(self):
        #prepare_data is called from a single process (e.g. GPU 0). Do not use it to assign state (self.x = y).
        _ = AutoTokenizer.from_pretrained(self.model_name_or_path)

    def setup(self, stage=None):
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name_or_path)
        if self.dataframe is None:
            self.dataframe = pd.read_csv(self.data_file_path)
        
        self.dataframe = self.dataframe[self.dataframe.txt.notna()].copy()
        
        cats = self.dataframe.category.apply(lambda x: x.split('|'))
        cat2cnt = Counter((j for i in cats for j in i))
        i2cat = sorted(k for k,v in cat2cnt.items() if v>self.min_products_for_category)
        cat2i = {v:k for k,v in enumerate(i2cat)}
        self.num_classes = len(i2cat)
        self.i2cat, self.cat2i = i2cat, cat2i
        
        ys = np.zeros((len(self.dataframe), len(i2cat)))
        for i,cats in enumerate(self.dataframe.category):
            idx_pos = [cat2i[cat] for cat in cats.split('|') if cat in cat2i]
            ys[i,idx_pos] = 1
        
        msk_val = self.dataframe.is_validation==1
        self.df_trn = self.dataframe[~msk_val]
        self.df_val = self.dataframe[msk_val]
        idx_trn = np.where(~msk_val)[0]
        idx_val = np.where(msk_val)[0]
        self.ys_trn, self.ys_val = ys[idx_trn], ys[idx_val]
        
        txt = self.dataframe.txt.values
        self.train_dataset = mk_ds(list(self.df_trn.txt), self.tokenizer, self.max_seq_length, self.ys_trn)
        self.eval_dataset  = mk_ds(list(self.df_val.txt), self.tokenizer, self.max_seq_length, self.ys_val)
        
    
    def train_dataloader(self):
        return DataLoader(
            self.train_dataset,
            batch_size=self.train_batch_size,
        )

    def val_dataloader(self):
        return DataLoader(
            self.eval_dataset,
            batch_size=self.val_batch_size,
        )

# model

In [9]:
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoModel


def getaccu(logits, ys):
    return ((logits>0.).int() == ys).float().mean()

class PCModel(pl.LightningModule):
    def __init__(self, model_name_or_path, num_classes, learning_rate, adam_beta1, adam_beta2, adam_epsilon):
        super().__init__()
        self.save_hyperparameters()
        self.model_name_or_path = model_name_or_path
        self.bert = AutoModel.from_pretrained(self.model_name_or_path)
        self.num_classes = num_classes
        self.W = nn.Linear(self.bert.config.hidden_size, self.num_classes)

    def prepare_data(self):
        #prepare_data is called from a single process (e.g. GPU 0). Do not use it to assign state (self.x = y).
        _ = AutoModel.from_pretrained(self.model_name_or_path)

    def forward(self, input_ids, attention_mask):
        h = self.bert(input_ids, attention_mask)['last_hidden_state']
        h_cls = h[:, 0]
        return self.W(h_cls)
    
    def training_step(self, batch, batch_idx):
        input_ids, attention_mask, ys = batch    
        logits = self(input_ids, attention_mask)
        loss = F.binary_cross_entropy_with_logits(logits, ys)
        accu = getaccu(logits, ys)
        self.log('train_loss', loss, on_epoch=True)
        self.log('train_accu', accu, on_epoch=True)
        return loss

    def validation_step(self, batch, batch_idx):
        input_ids, attention_mask, ys = batch    
        logits = self(input_ids, attention_mask)
        loss = F.binary_cross_entropy_with_logits(logits, ys)
        accu = getaccu(logits, ys)
        self.log('valid_loss', loss, on_step=False, sync_dist=True)
        self.log('valid_accu', accu, on_step=False, sync_dist=True)
    
    def configure_optimizers(self):
        optimizer = AdamW(self.parameters(),
                          self.hparams.learning_rate,
                          betas=(self.hparams.adam_beta1,
                                 self.hparams.adam_beta2),
                          eps=self.hparams.adam_epsilon,)
        return optimizer    
    
    @staticmethod
    def add_model_specific_args(parent_parser):
        parser = ArgumentParser(parents=[parent_parser], add_help=False)
        parser.add_argument('--learning_rate', type=float, default=5e-5)
        parser.add_argument('--adam_beta1', type=float, default=0.9)
        parser.add_argument('--adam_beta2', type=float, default=0.999)
        parser.add_argument('--adam_epsilon', type=float, default=1e-8)
        return parser

# train

In [10]:
parser = ArgumentParser()

parser.add_argument('--model_name_or_path', type=str,
                    default="distilbert-base-cased")
parser.add_argument('--max_seq_length', type=int, default=512)
parser.add_argument('--min_products_for_category', type=int, default=100)
parser.add_argument('--train_batch_size', type=int, default=32)
parser.add_argument('--val_batch_size', type=int, default=64)

parser = pl.Trainer.add_argparse_args(parser)
parser = PCModel.add_model_specific_args(parser)

args = parser.parse_args([
    '--default_root_dir', p_out,
])


data_module = PCDataModule(
    model_name_or_path=args.model_name_or_path,
#     data_file_path=f'{HOME}/data/data_sample__{prfx_prp}.csv',
    data_file_path=DATA2USE,
    min_products_for_category=args.min_products_for_category,
    max_seq_length=args.max_seq_length,
    train_batch_size=args.train_batch_size,
    val_batch_size=args.val_batch_size,
)

data_module.prepare_data()

In [11]:
%%time
data_module.setup()

CPU times: user 9 s, sys: 555 ms, total: 9.56 s
Wall time: 2.62 s


In [12]:
pcmodel = PCModel(
    model_name_or_path=args.model_name_or_path,
    num_classes= data_module.num_classes,
    learning_rate=args.learning_rate,
    adam_beta1=args.adam_beta1,
    adam_beta2=args.adam_beta2,
    adam_epsilon=args.adam_epsilon,
)
pcmodel.prepare_data()

In [13]:
dl = data_module.val_dataloader()
n=0
for dat in dl:
    input_ids, attention_mask, ys = dat
    logits = pcmodel(input_ids, attention_mask)
    accu = getaccu(logits, ys)
    print(accu.item)
    n+=1
    if n>3: break

tensor(0.5695)
tensor(0.5622)
tensor(0.5638)
tensor(0.5641)


## run train

In [14]:
pl.seed_everything(1234)
trainer = pl.Trainer.from_argparse_args(args, 
#                                         limit_train_batches=10, limit_val_batches=5, 
                                        max_epochs=1,
#                                         fast_dev_run=True, 
                                        log_gpu_memory=True, gpus=1)


Global seed set to 1234
GPU available: True, used: True
TPU available: None, using: 0 TPU cores


In [15]:
trainer.fit(pcmodel, data_module)


  | Name | Type            | Params
-----------------------------------------
0 | bert | DistilBertModel | 65.2 M
1 | W    | Linear          | 37.7 K
-----------------------------------------
65.2 M    Trainable params
0         Non-trainable params
65.2 M    Total params
260.914   Total estimated model params size (MB)
/data/anaconda3/envs/product-category/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


/data/anaconda3/envs/product-category/lib/python3.7/site-packages/pytorch_lightning/core/step_result.py:148: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  value = torch.tensor(value, device=device, dtype=torch.float)
/data/anaconda3/envs/product-category/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


1

## tensorboard

In [16]:
!ls {p_out}/lightning_logs/

version_0


# eval

In [19]:
dl = data_module.val_dataloader()
n=0
for dat in dl:
    input_ids, attention_mask, ys = dat

    logits = pcmodel(input_ids, attention_mask)

    accu = getaccu(logits, ys)
    print(accu.item())
    n+=1
    if n>3: break

0.9732142686843872
0.9693877696990967
0.969706654548645
0.9693877696990967


## demo

In [84]:
data_module.ys_trn.mean()

0.03892962102051056

In [85]:
data_module.ys_val.mean()

0.03873355509604723

In [81]:
df_trn = data_module.df_trn
i2cat = data_module.i2cat
tokenizer = data_module.tokenizer
max_seq_length = data_module.max_seq_length


row = df_trn.sample()
display(row)
txt = list(row.txt.values)
input_ids, attention_mask = mk_tensors(txt, tokenizer, max_seq_length)
logits = pcmodel(input_ids, attention_mask)
print(sorted(row.category.values[0].split('|')))
[i2cat[i] for i in np.where(logits>0)[0]]

,category,description,title,brand,feature,asin,domain,txt,is_validation
6115,Home & Kitchen|Home Dcor|Candles & Holders|Can...,Advent is a time of patience and preparation. ...,Advent Candle Kit,Illuminated Ink,Each kit makes four 8 tapered candles (3 viole...,B001E1OLWW,Home_and_Kitche,Advent Candle Kit Illuminated Ink Advent is a ...,0.0


['Candles', 'Candles & Holders', 'Home & Kitchen', 'Home Dcor', 'Seasonal Celebration Candles', 'Specialty Candles']


[]

In [36]:
df_val = data_module.df_val
i2cat = data_module.i2cat
tokenizer = data_module.tokenizer
max_seq_length = data_module.max_seq_length


row = df_val.sample()
display(row)
txt = list(row.txt.values)
input_ids, attention_mask = mk_tensors(txt, tokenizer, max_seq_length)
logits = pcmodel(input_ids, attention_mask)
print(sorted(row.category.values[0].split('|')))
[i2cat[i] for i in np.where(logits>0)[0]]

# fin

In [18]:
!nvidia-smi

Sat Mar  6 14:01:02 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.100      Driver Version: 440.100      CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:1E.0 Off |                    0 |
| N/A   53C    P0    44W / 300W |   1930MiB / 16160MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  